In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphEmbedderObjective import SubgraphEmbeddingObjective, \
    SubgraphMatchingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist
from SourceCodeTools.models.graph.train.objectives.SCAAClassifierObjetive import SCAAClassifierObjective


import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
tokenizer_path = "sentencepiece_bpe.model"

data_path = "./examples/one_vs_10/with_ast"
subgraph_partition = join(data_path, "subgraph_partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

In [4]:
unpersist(subgraph_partition)

,id,train_mask,val_mask,test_mask
0,888,True,False,False
1,887,True,False,False
2,644,True,False,False
3,5341,False,False,True
4,5318,True,False,False
...,...,...,...,...
85,5314,True,False,False
86,957,True,False,False
87,5311,False,True,False
88,454,True,False,False


In [5]:
unpersist(join(data_path, "common_edges.json.bz2"), nrows=10)

,id,type,source_node_id,target_node_id,file_id,mentioned_in
0,0,subword,7025,16744,888,NaN
1,1,subword,17637,16744,888,NaN
2,2,subword,3966,16744,888,NaN
3,3,module,16744,9058,888,5968.0
4,4,module_rev,9058,16744,888,5968.0
5,5,subword,13875,13928,888,NaN
6,6,name,13928,3104,888,5968.0
7,7,name_rev,3104,13928,888,5968.0
8,8,names,3104,9058,888,5968.0
9,9,names_rev,9058,3104,888,5968.0


In [6]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    measure_scores=True,
    dilate_scores=1,
    epochs=100,
)

In [7]:
save_config(config, "var_misuse_tiny.yaml")

In [8]:
config

{'DATASET': {'data_path': './examples/one_vs_10/with_ast',
  'train_frac': 0.9,
  'filter_edges': None,
  'min_count_for_objectives': 5,
  'self_loops': False,
  'use_node_types': False,
  'use_edge_types': False,
  'no_global_edges': False,
  'remove_reverse': False,
  'custom_reverse': None,
  'restricted_id_pool': None,
  'random_seed': None,
  'subgraph_id_column': 'file_id',
  'subgraph_partition': './examples/one_vs_10/with_ast/subgraph_partition.json.bz2'},
 'TRAINING': {'model_output_dir': './examples/one_vs_10/with_ast',
  'pretrained': None,
  'pretraining_phase': 0,
  'sampling_neighbourhood_size': 10,
  'neg_sampling_factor': 3,
  'use_layer_scheduling': False,
  'schedule_layers_every': 10,
  'elem_emb_size': 100,
  'embedding_table_size': 200000,
  'epochs': 100,
  'batch_size': 128,
  'learning_rate': 0.001,
  'objectives': 'subgraph_clf',
  'save_each_epoch': False,
  'save_checkpoints': True,
  'early_stopping': False,
  'early_stopping_tolerance': 20,
  'force_w2v_ns'

# Create Dataset

In [9]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

/home/alina/workspace/scaa/method-embedding/SourceCodeTools/code/data/dataset/Dataset.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.nodes = self.nodes.append(new_nodes, ignore_index=True)
/home/alina/workspace/scaa/method-embedding/SourceCodeTools/code/data/dataset/Dataset.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.edges = self.edges.append(new_edges, ignore_index=True)


# Declare target loading function (labels)

In [10]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "user"]].rename({"id": "src", "user": "dst"}, axis=1)

In [11]:
load_labels()

,src,dst
0,888,Itachi_uchiha
1,887,Itachi_uchiha
2,644,Fatality2412
3,5341,Fatality2412
4,5318,Chuckminster
...,...,...
85,5314,BarryFam
86,957,JoelSpringer
87,5311,Avi_Sri
88,454,Chuckminster


One or several objectives could be used

In [12]:
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="SCAAMatching",
                objective_class=SCAAClassifierObjective,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

In [13]:
# %tensorboard --logdir data
from tensorboard import program

tracking_address = './examples/one_vs_10/' # the path of your log file.

tb = program.TensorBoard()
tb.configure(argv=[None, '--logdir', tracking_address])
url = tb.launch()
print(f"Tensorflow listening on {url}")


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

2022-05-25 21:12:18.830061: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-25 21:12:18.830213: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Tensorflow listening on http://localhost:6006/


In [14]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Precompute Target Embeddings:   0%|                                                                                      | 0/1 [00:00<?, ?it/s]

Number of nodes 17892


Epoch 1:   0%|                                                                                                           | 0/1 [00:00<?, ?it/s]

Epoch: 0, Time: 6 s
{'Accuracy/test/SCAAMatching_': 0.5,
 'Accuracy/train/SCAAMatching_': 0.5,
 'Accuracy/train_avg/SCAAMatching': 0.5,
 'Accuracy/val/SCAAMatching_': 0.5,
 'Loss/test/SCAAMatching_': 0.6015158295631409,
 'Loss/train/SCAAMatching_': 0.5919040441513062,
 'Loss/train_avg/SCAAMatching': 0.5919040441513062,
 'Loss/val/SCAAMatching_': 0.5973190069198608,
 'hits@1/test/SCAAMatching_': 0.05,
 'hits@1/val/SCAAMatching_': 0.1,
 'hits@10/test/SCAAMatching_': 0.11875,
 'hits@10/val/SCAAMatching_': 0.125,
 'hits@3/test/SCAAMatching_': 0.08333333333333333,
 'hits@3/val/SCAAMatching_': 0.1,
 'hits@5/test/SCAAMatching_': 0.09,
 'hits@5/val/SCAAMatching_': 0.1,
 'map/test/SCAAMatching_': 0.1800709349374379,
 'map/val/SCAAMatching_': 0.17977364167777116,
 'mr/test/SCAAMatching_': 23.5,
 'mr/val/SCAAMatching_': 23.5,
 'mrr/test/SCAAMatching_': 0.15123215602874715,
 'mrr/val/SCAAMatching_': 0.15123215602874723,
 'ndcg@1/test/SCAAMatching_': 0.08888888888888888,
 'ndcg@1/val/SCAAMatching_'

Epoch 2:   0%|                                                                                                           | 0/1 [00:00<?, ?it/s]

...

Precompute Target Embeddings:   0%|                                                                                      | 0/1 [00:00<?, ?it/s]

Epoch: 99, Time: 8 s
{'Accuracy/test/SCAAMatching_': 0.55,
 'Accuracy/train/SCAAMatching_': 0.5666666666666667,
 'Accuracy/train_avg/SCAAMatching': 0.5666666666666667,
 'Accuracy/val/SCAAMatching_': 0.5,
 'Loss/test/SCAAMatching_': 0.5510892868041992,
 'Loss/train/SCAAMatching_': 0.5127081274986267,
 'Loss/train_avg/SCAAMatching': 0.5127081274986267,
 'Loss/val/SCAAMatching_': 0.673780083656311,
 'hits@1/test/SCAAMatching_': 0.05,
 'hits@1/val/SCAAMatching_': 0.1,
 'hits@10/test/SCAAMatching_': 0.11875,
 'hits@10/val/SCAAMatching_': 0.125,
 'hits@3/test/SCAAMatching_': 0.08333333333333333,
 'hits@3/val/SCAAMatching_': 0.1,
 'hits@5/test/SCAAMatching_': 0.09,
 'hits@5/val/SCAAMatching_': 0.1,
 'map/test/SCAAMatching_': 0.1800709349374379,
 'map/val/SCAAMatching_': 0.17977364167777116,
 'mr/test/SCAAMatching_': 23.5,
 'mr/val/SCAAMatching_': 23.5,
 'mrr/test/SCAAMatching_': 0.15123215602874715,
 'mrr/val/SCAAMatching_': 0.15123215602874723,
 'ndcg@1/test/SCAAMatching_': 0.088888888888888

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.32it/s]

Final eval: hits@1/val/SCAAMatching_final: 0.1, hits@3/val/SCAAMatching_final: 0.1, hits@5/val/SCAAMatching_final: 0.1, hits@10/val/SCAAMatching_final: 0.125, ndcg@1/val/SCAAMatching_final: 0.08888888888888889, ndcg@3/val/SCAAMatching_final: 0.08888888888888889, ndcg@5/val/SCAAMatching_final: 0.08888888888888888, ndcg@10/val/SCAAMatching_final: 0.10215646036989276, mr/val/SCAAMatching_final: 23.5, mrr/val/SCAAMatching_final: 0.15123215602874723, map/val/SCAAMatching_final: 0.17977364167777116, scoring_time/val/SCAAMatching_final: 0.008248090744018555, Loss/val/SCAAMatching_final: 0.7112621068954468, Accuracy/val/SCAAMatching_final: 0.5, hits@1/test/SCAAMatching_final: 0.05, hits@3/test/SCAAMatching_final: 0.08333333333333333, hits@5/test/SCAAMatching_final: 0.09, hits@10/test/SCAAMatching_final: 0.11875, ndcg@1/test/SCAAMatching_final: 0.08888888888888888, ndcg@3/test/SCAAMatching_final: 0.08888888888888888, ndcg@5/test/SCAAMatching_final: 0.08888888888888885, ndcg@10/test/SCAAMatching

(<__main__.Trainer at 0x7fcfc26af250>,
 {'hits@1/val/SCAAMatching_final': 0.1,
  'hits@3/val/SCAAMatching_final': 0.1,
  'hits@5/val/SCAAMatching_final': 0.1,
  'hits@10/val/SCAAMatching_final': 0.125,
  'ndcg@1/val/SCAAMatching_final': 0.08888888888888889,
  'ndcg@3/val/SCAAMatching_final': 0.08888888888888889,
  'ndcg@5/val/SCAAMatching_final': 0.08888888888888888,
  'ndcg@10/val/SCAAMatching_final': 0.10215646036989276,
  'mr/val/SCAAMatching_final': 23.5,
  'mrr/val/SCAAMatching_final': 0.15123215602874723,
  'map/val/SCAAMatching_final': 0.17977364167777116,
  'scoring_time/val/SCAAMatching_final': 0.008248090744018555,
  'Loss/val/SCAAMatching_final': 0.7112621068954468,
  'Accuracy/val/SCAAMatching_final': 0.5,
  'hits@1/test/SCAAMatching_final': 0.05,
  'hits@3/test/SCAAMatching_final': 0.08333333333333333,
  'hits@5/test/SCAAMatching_final': 0.09,
  'hits@10/test/SCAAMatching_final': 0.11875,
  'ndcg@1/test/SCAAMatching_final': 0.08888888888888888,
  'ndcg@3/test/SCAAMatching_